# Portfolio Composition Analysis

In [7]:
import sys
import os

# Add src directory to path
sys.path.insert(0, os.path.abspath('../'))

import json
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

from core.portfolio import analyze_portfolio_composition
from core.etf_analyzer import analyze_portfolio_with_lookthrough, get_etf_info
from config import PORTFOLIO_FILE

pd.set_option('display.precision', 2)
pd.set_option('display.float_format', '{:.2f}'.format)


## 1. Load Portfolio Configuration


In [8]:
with open(f'../{PORTFOLIO_FILE}', 'r') as f:
    portfolio = json.load(f)

print("Current Portfolio Configuration:")
print("=" * 50)
for item in portfolio:
    ticker = item.get("ticker", "N/A")
    type_ = item.get("type", "N/A")
    label_value = None
    label_kind = None

    # Use whichever key is present and show value appropriately
    if "units" in item:
        label_value = item["units"]
        label_kind = "units"
        value_fmt = f"{label_value:8.2f}"
    elif "weight" in item:
        label_value = item["weight"]
        label_kind = "weight"
        value_fmt = f"{label_value:8.4f}"
    elif "percentage" in item:
        label_value = item["percentage"]
        label_kind = "percentage"
        value_fmt = f"{label_value:8.2f}%"
    else:
        label_value = "N/A"
        label_kind = ""
        value_fmt = f"{label_value:>8}"

    print(f"{ticker:6} - {value_fmt} {label_kind} ({type_})")
print("=" * 50)


Current Portfolio Configuration:
URTH   -   0.3649 weight (etf)
IMAE.AS -   0.3649 weight (etf)
XEOND.XD -   0.1520 weight (etf)
CEBE.DE -   0.0405 weight (etf)


## 2. Asset Allocation Analysis


In [9]:
# Generate asset and sector allocation visualizations
fig_asset, fig_sector = analyze_portfolio_composition(f'../{PORTFOLIO_FILE}', lookup_etf_holdings = True)

if fig_asset:
    fig_asset.show()

if fig_sector:
    fig_sector.show()


TypeError: analyze_portfolio_composition() got an unexpected keyword argument 'lookup_etf_holdings'

## 3. Detailed Holdings Table


In [ ]:
import yfinance as yf

# Create detailed holdings table
holdings_data = []

for item in portfolio:
    ticker = item['ticker']
    item_type = item.get('type', '').lower()

    try:
        ticker_obj = yf.Ticker(ticker)
        price = ticker_obj.fast_info['lastPrice']

        # Assume "units" OR "weight", not both
        if "units" in item:
            quantity_label = "Units"
            quantity = item["units"]
            market_value = quantity * price
        elif "weight" in item:
            quantity_label = "Weight"
            quantity = item["weight"]
            market_value = price  # Will not be used
        else:
            quantity_label = "Units"
            quantity = 0
            market_value = 0

        sector = ticker_obj.info.get('sector', 'N/A') if item_type == 'stock' else 'ETF'

        holdings_data.append({
            'Ticker': ticker,
            'Type': item_type.upper(),
            quantity_label: quantity,
            'Price': price,
            'Market Value': market_value,
            'Sector': sector,
            'Raw_Weight': item.get("weight", None)
        })

    except Exception as e:
        print(f"Error fetching {ticker}: {e}")

df_holdings = pd.DataFrame(holdings_data)

# If portfolio uses "units"
if "Units" in df_holdings.columns:
    total_value = df_holdings['Market Value'].sum()
else:
    df_holdings["Market Value"] = df_holdings["Raw_Weight"].astype(float) * df_holdings["Price"]
    total_value = df_holdings["Market Value"].sum()

# Return the dataframe and the total market value
df_holdings, total_value



Weighted Portfolio Value (arbitrary, for display): $122.41

Detailed Holdings:


,Ticker,Type,Weight,Price,Market Value,Sector,Weight (%)
0,URTH,ETF,0.3649,$182.56,$66.62,ETF,36.49%
1,IMAE.AS,ETF,0.3649,$90.79,$33.13,ETF,36.49%
2,XEOND.XD,ETF,0.1520,$147.65,$22.44,ETF,15.20%
3,CEBE.DE,ETF,0.0405,$5.49,$0.22,ETF,4.05%


## 4. ETF Look-Through Analysis

This section attempts to analyze the underlying holdings of ETFs in your portfolio to understand indirect exposure.


In [ ]:
# Analyze portfolio with ETF look-through
lookthrough_df = analyze_portfolio_with_lookthrough(portfolio, max_etf_holdings=15)

if not lookthrough_df.empty:
    print("\nTotal Exposure (Direct + Indirect via ETFs):")
    print("=" * 70)
    
    display(lookthrough_df.style.format({
        'Exposure_Value': '${:,.2f}',
        'Portfolio_Weight': '{:.2%}'
    }))
    
    # Visualize combined exposure
    fig = px.bar(
        lookthrough_df.head(15),
        x='Ticker',
        y='Portfolio_Weight',
        title='Total Exposure Including ETF Look-Through (Top 15)',
        labels={'Portfolio_Weight': 'Weight (%)', 'Ticker': 'Asset'}
    )
    fig.update_yaxes(tickformat='.1%')
    fig.show()
else:
    print("ETF holdings data not available for look-through analysis.")


Error retrieving holdings for VOO: 'FundsData' object has no attribute 'get'

Total Exposure (Direct + Indirect via ETFs):


,Ticker,Exposure_Value,Portfolio_Weight
4,VOO,"$30,844.50",69.85%
2,MSFT,"$7,452.30",16.88%
0,AAPL,"$2,684.70",6.08%
1,GOOG,"$2,237.60",5.07%
3,NVDA,$940.75,2.13%


## 5. ETF Details

Detailed information about each ETF in the portfolio.


In [ ]:
# Get details for all ETFs in portfolio
etf_items = [item for item in portfolio if item.get('type') == 'etf']

for etf_item in etf_items:
    ticker = etf_item['ticker']
    print(f"\n{'='*70}")
    print(f"ETF: {ticker}")
    print(f"{'='*70}")
    
    etf_info = get_etf_info(ticker)
    
    for key, value in etf_info.items():
        print(f"{key.replace('_', ' '):.<30} {value}")



ETF: VOO
Name.......................... Vanguard S&P 500 ETF
Category...................... Large Blend
Total Assets.................. 1409023475712
Expense Ratio................. N/A
YTD Return.................... 14.80406
Three Year Return............. 0.24263781
Five Year Return.............. 0.16042851


## 6. Sector Breakdown

Comprehensive sector exposure across all holdings (excluding ETFs).


In [ ]:
# Sector breakdown for stocks only
stock_holdings = df_holdings[df_holdings['Type'] == 'STOCK'].copy()

if not stock_holdings.empty:
    sector_summary = stock_holdings.groupby('Sector').agg({
        'Market Value': 'sum',
        'Ticker': 'count'
    }).reset_index()
    sector_summary.columns = ['Sector', 'Market Value', 'Count']
    sector_summary['Weight (%)'] = (sector_summary['Market Value'] / stock_holdings['Market Value'].sum() * 100)
    sector_summary = sector_summary.sort_values('Weight (%)', ascending=False)
    
    print("\nSector Exposure (Stocks Only):")
    display(sector_summary.style.format({
        'Market Value': '${:,.2f}',
        'Weight (%)': '{:.2f}%'
    }))



Sector Exposure (Stocks Only):


,Sector,Market Value,Count,Weight (%)
1,Technology,"$11,077.75",3,83.20%
0,Communication Services,"$2,237.60",1,16.80%


## Summary

This notebook provides a comprehensive view of your portfolio composition. Key insights:

1. **Asset Allocation**: Shows how your capital is distributed across different assets
2. **Sector Exposure**: Identifies concentration in specific sectors
3. **ETF Look-Through**: Reveals indirect exposure through ETF holdings

**Next Steps:**

- Adjust holdings in `config_ticker.json` to rebalance
- Review sector concentration for diversification
- Use the Index Simulation notebook to analyze market scenarios

